In [ ]:
import sys
sys.path.append('/home/lo276838/Modèles/mri-dl-recon/src/mri_dlrecon')

In [ ]:
from fastmri.data import transforms as T
import torch
import numpy as np
from matplotlib import pyplot as plt
from torch.autograd import Variable

from utils_torch import virtual_coil_reconstruction, load_and_transform, create_zero_filled_reconstruction
from fastmri.data.subsample import create_mask_for_mask_type
from fastmri.models.unet import Unet


In [ ]:
plt.rcParams['figure.figsize'] = (7, 7)
# plt.rcParams['image.cmap'] = 'gray'

In [ ]:
file_path = "/volatile/FastMRI/brain_multicoil_train/multicoil_train/file_brain_AXT1POST_201_6002780.h5"

kspace_multicoil = load_and_transform(file_path)
images_multicoil = torch.fft.fftshift(torch.fft.ifft2(kspace_multicoil))

image = virtual_coil_reconstruction(images_multicoil)
kspace = torch.fft.fft2(image)

mask_type = "magic_fraction"
center_fractions = [0.08, 0.04]
accelerations = [4, 8]
mask = create_mask_for_mask_type(mask_type, center_fractions, accelerations)

zero_filled_reconstruction = torch.fft.fftshift(create_zero_filled_reconstruction(mask,kspace))


input = zero_filled_reconstruction
target = image.abs().unsqueeze(1)

In [ ]:
plt.subplot(1,2,1)
plt.imshow(np.abs(input[0, 0, :,: ].numpy()))
plt.title("Input")

plt.subplot(1,2,2)
plt.imshow(target[0,0, :,: ].numpy())
plt.title("Target")

plt.tight_layout()
plt.show()